In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,accuracy_score
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from data_treat.get_ready_data import get_data

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
data = get_data()

In [10]:
data.head()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
44563,5.473170,0.270352,0.995224,1.0,0.0,0.0,1.0,0.0
35891,5.539120,8.436056,2.351681,1.0,0.0,1.0,0.0,0.0
83053,0.768174,11.256669,1.469458,0.0,0.0,0.0,1.0,0.0
47470,2.221584,0.058871,0.941695,1.0,1.0,0.0,1.0,0.0
16841,4.235375,10.563239,0.476625,1.0,0.0,0.0,1.0,0.0


In [11]:
data.describe()

,distance_from_home,distance_from_last_transaction,ratio_to_median_purchase_price,repeat_retailer,used_chip,used_pin_number,online_order,fraud
count,174806.000000,174806.000000,174806.000000,174806.000000,174806.000000,174806.000000,174806.000000,174806.000000
mean,44.596086,8.502264,3.713906,0.881383,0.308216,0.057916,0.784327,0.500000
std,104.753108,36.772105,4.755045,0.323338,0.461758,0.233585,0.411290,0.500001
min,0.021322,0.000407,0.011373,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.137164,0.309630,0.690949,1.000000,0.000000,0.000000,1.000000,0.000000
50%,11.754798,1.063041,2.006738,1.000000,0.000000,0.000000,1.000000,0.500000
75%,37.594545,3.811184,5.236790,1.000000,1.000000,0.000000,1.000000,1.000000
max,10632.723672,2160.499922,266.689692,1.000000,1.000000,1.000000,1.000000,1.000000


Based from the describe above, we can see that we need to normalize the data

In [12]:
data.dtypes

distance_from_home                float64
distance_from_last_transaction    float64
ratio_to_median_purchase_price    float64
repeat_retailer                   float64
used_chip                         float64
used_pin_number                   float64
online_order                      float64
fraud                             float64
dtype: object

We can see from above that the data is all in int/float format. Now we check if there are any null value

In [13]:
data.isnull().sum()

distance_from_home                0
distance_from_last_transaction    0
ratio_to_median_purchase_price    0
repeat_retailer                   0
used_chip                         0
used_pin_number                   0
online_order                      0
fraud                             0
dtype: int64

In [ ]:
df['fraud'].plot(kind='hist')

In [15]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20,random_state=42, stratify=y)

In [ ]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [ ]:
def gridSearch(X,y):

    param_dict = {
    "criterion": ['gini', 'entropy'],
    "max_depth" : np.arange(1,10,2),
    "splitter": ['best', 'random'],
    }
    model = DecisionTreeClassifier(random_state=42)
    gv = GridSearchCV(model,param_grid=param_dict,cv=10)
    gv.fit(X,y)
    return gv.best_score_,gv.best_params_


In [ ]:
gridSearch(X_train,y_train)

In [ ]:
model = DecisionTreeClassifier(random_state=42,
                               criterion = 'entropy',
                               max_depth =  7,
                               splitter =  'best')

In [ ]:
model.fit(X_train,y_train)
prediction = model.predict(X_test)


In [ ]:
print ("Accuracy : ",accuracy_score(y_test,prediction)*100)
print(classification_report(y_test, prediction))